Library imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

import utilities as utils

/Users/e35547/Library/Python/3.9/lib/python/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Loads the datacube

In [2]:
data = utils.load_dataset()
data.head()

,"ï»¿""H3_Address""",H3_Resolution,H3_Geometry,Longitude_EPSG4326,Latitude_EPSG4326,Continent_Majority,Continent_Minority,Country_Majority,Country_Minority,Province_Majority,...,Litmod_Density_Asthenosphere,Litmod_Density_Crust,Litmod_Density_Lithosphere,Crust1_Type,Crust1_CrustalThickness,Crust1_SedimentThickness,Training_MVT_Deposit,Training_MVT_Occurrence,Training_CD_Deposit,Training_CD_Occurrence
0,8712e579bffffff,7,"POLYGON ((-115.0314 54.5077, -115.0393 54.4961...",-115.018142,54.497221,North America,North America,Canada,Canada,Alberta,...,3480.580078,2891.260254,3337.300049,island arc,-38.450497,2991.459961,Absent,Absent,Absent,Absent
1,8712e579affffff,7,"POLYGON ((-115.0658 54.51706, -115.0737 54.505...",-115.052542,54.506590,North America,North America,Canada,Canada,Alberta,...,3480.580078,2891.260010,3337.300293,island arc,-38.430000,3000.000244,Absent,Absent,Absent,Absent
2,8712e56b4ffffff,7,"POLYGON ((-115.0604 54.49501, -115.0682 54.483...",-115.047107,54.484541,North America,North America,Canada,Canada,Alberta,...,3480.580078,2891.259766,3337.300049,island arc,-38.430000,3000.000000,Absent,Absent,Absent,Absent
3,8712e56b5ffffff,7,"POLYGON ((-115.026 54.48564, -115.0338 54.4740...",-115.012729,54.475169,North America,North America,Canada,Canada,Alberta,...,3480.580078,2891.260010,3337.300049,island arc,-38.591599,2932.666504,Absent,Absent,Absent,Absent
4,8712e56a6ffffff,7,"POLYGON ((-114.997 54.49832, -115.0049 54.4867...",-114.983753,54.487840,North America,North America,Canada,Canada,Alberta,...,3480.580078,2891.260010,3337.300049,island arc,-39.815273,2422.801758,Absent,Absent,Absent,Absent


In [3]:
# modifies presence / absence columns to boolean - geology properties
data["Geology_Dictionary_Alkalic"] = data["Geology_Dictionary_Alkalic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Anatectic"] = data["Geology_Dictionary_Anatectic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Calcareous"] = data["Geology_Dictionary_Calcareous"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Carbonaceous"] = data["Geology_Dictionary_Carbonaceous"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Cherty"] = data["Geology_Dictionary_Cherty"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_CoarseClastic"] = data["Geology_Dictionary_CoarseClastic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Evaporitic"] = data["Geology_Dictionary_Evaporitic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Felsic"] = data["Geology_Dictionary_Felsic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_FineClastic"] = data["Geology_Dictionary_FineClastic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Gneissose"] = data["Geology_Dictionary_Gneissose"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Igneous"] = data["Geology_Dictionary_Igneous"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Intermediate"] = data["Geology_Dictionary_Intermediate"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Pegmatitic"] = data["Geology_Dictionary_Pegmatitic"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_RedBed"] = data["Geology_Dictionary_RedBed"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Schistose"] = data["Geology_Dictionary_Schistose"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_Sedimentary"] = data["Geology_Dictionary_Sedimentary"].apply(lambda x: True if x == "Present" else False)
data["Geology_Dictionary_UltramaficMafic"] = data["Geology_Dictionary_UltramaficMafic"].apply(lambda x: True if x == "Present" else False)
# modifies presence / absence columns to boolean - labels
data["Training_MVT_Deposit"] = data["Training_MVT_Deposit"].apply(lambda x: True if x == "Present" else False)
data["Training_MVT_Occurrence"] = data["Training_MVT_Occurrence"].apply(lambda x: True if x == "Present" else False)
data["Training_CD_Deposit"] = data["Training_CD_Deposit"].apply(lambda x: True if x == "Present" else False)
data["Training_CD_Occurrence"] = data["Training_CD_Occurrence"].apply(lambda x: True if x == "Present" else False)

Selects the data /labels used for MVT WOE baseline

In [4]:
cols_dict = utils.load_features_dict(type='MVT', baseline='preferred')
data_filtered, cols = utils.extract_cols(data, cols_dict)

data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5164970 entries, 0 to 5164969
Data columns (total 27 columns):
 #   Column                                          Dtype  
---  ------                                          -----  
 0   H3_Geometry                                     object 
 1   Geology_Lithology_Majority                      object 
 2   Geology_Lithology_Minority                      object 
 3   Geology_Period_Maximum_Majority                 object 
 4   Geology_Period_Minimum_Majority                 object 
 5   Sedimentary_Dictionary                          bool   
 6   Igneous_Dictionary                              bool   
 7   Metamorphic_Dictionary                          bool   
 8   Seismic_LAB_Priestley                           float64
 9   Seismic_Moho                                    float64
 10  Gravity_GOCE_ShapeIndex                         float64
 11  Geology_Paleolatitude_Period_Minimum            float64
 12  Terrane_Proximity           

The following function finds all the neighbors and creates a new column "MVT_Deposit".
Original paper treats neighbors of polygons with "Training_MVT_Deposit=Present" and "Training_MVT_Occurrence=Present" as mineral present, "MVT_Deposit=Present" (note: now Deposit means - Deposit, Occurrence, or their neighbor). 

In [5]:
data_filtered = utils.neighbor_deposits(data_filtered, type='MVT')

In [6]:
print(data_filtered['MVT_Deposit'].value_counts())
print(data_filtered['MVT_Deposit_wNeighbors'].value_counts())

MVT_Deposit
False    5162943
True        2027
Name: count, dtype: int64
MVT_Deposit_wNeighbors
False    5155482
True        9488
Name: count, dtype: int64


In [7]:
labels_filtered = data_filtered['MVT_Deposit_wNeighbors']
data_filtered = data_filtered.drop(columns=['H3_Geometry', 'Training_MVT_Deposit', 'Training_MVT_Occurrence', 'MVT_Deposit', 'MVT_Deposit_wNeighbors'])
cols = cols[1:-2]

Clearly the dataset has MANY outliers, as reported in the paper

In [8]:
# ax = sns.boxplot(data=data_filtered, orient="h", palette="Set2")

In [9]:
data_filtered['Geology_Lithology_Majority'].value_counts()

Geology_Lithology_Majority
Sedimentary_Siliciclastic        1713721
Other_Unconsolidated             1641673
Igneous_Intrusive_Felsic          556014
Sedimentary_Chemical              466584
Igneous_Extrusive                 310350
Metamorphic_Gneiss                203916
Metamorphic_Gneiss_Paragneiss     146287
Metamorphic_Schist                 75355
Igneous_Intrusive_Mafic            51070
Name: count, dtype: int64

In [10]:
data_filtered = pd.get_dummies(data_filtered, columns=['Geology_Lithology_Majority','Geology_Lithology_Minority','Geology_Period_Maximum_Majority','Geology_Period_Minimum_Majority'], prefix=['Geology_Lithology_Majority','Geology_Lithology_Minority','Geology_Period_Maximum_Majority','Geology_Period_Minimum_Majority'])
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5164970 entries, 0 to 5164969
Data columns (total 78 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   Sedimentary_Dictionary                                    bool   
 1   Igneous_Dictionary                                        bool   
 2   Metamorphic_Dictionary                                    bool   
 3   Seismic_LAB_Priestley                                     float64
 4   Seismic_Moho                                              float64
 5   Gravity_GOCE_ShapeIndex                                   float64
 6   Geology_Paleolatitude_Period_Minimum                      float64
 7   Terrane_Proximity                                         float64
 8   Geology_PassiveMargin_Proximity                           float64
 9   Geology_BlackShale_Proximity                              float64
 10  Geology_Fault_Proximity       

We can remove these outliers

In [11]:
data_filtered = utils.tukey_remove_outliers(data_filtered)
# ax = sns.boxplot(data=data_filtered, orient="h", palette="Set2")

There are also many NaNs in the data, these can be "imputed" with the mean value.

In [12]:
print(data_filtered.isna().sum())

Sedimentary_Dictionary                              0
Igneous_Dictionary                                  0
Metamorphic_Dictionary                              0
Seismic_LAB_Priestley                               0
Seismic_Moho                                     1307
                                                 ... 
Geology_Period_Minimum_Majority_Pennsylvanian       0
Geology_Period_Minimum_Majority_Permian             0
Geology_Period_Minimum_Majority_Quaternary          0
Geology_Period_Minimum_Majority_Silurian            0
Geology_Period_Minimum_Majority_Triassic            0
Length: 78, dtype: int64


In [13]:
data_filtered = utils.impute_nans(data_filtered)
print(data_filtered.isna().sum())

Sedimentary_Dictionary                           0
Igneous_Dictionary                               0
Metamorphic_Dictionary                           0
Seismic_LAB_Priestley                            0
Seismic_Moho                                     0
                                                ..
Geology_Period_Minimum_Majority_Pennsylvanian    0
Geology_Period_Minimum_Majority_Permian          0
Geology_Period_Minimum_Majority_Quaternary       0
Geology_Period_Minimum_Majority_Silurian         0
Geology_Period_Minimum_Majority_Triassic         0
Length: 78, dtype: int64


Finally, it can be observed the above data is not "normalized", we should make features standard scores / z-scores

In [14]:
data_filtered = utils.normalize_df(data_filtered)
# ax = sns.boxplot(data=data_filtered, orient="h", palette="Set2")
# print("(note remaining outliers above were within the Tukey fences calculated over ALL the data)")

Appending 'target' and 'Latitude_EPSG4326' columns to data_filtered

In [15]:
data_filtered['target'] = labels_filtered
data_filtered['Latitude_EPSG4326'] = data['Latitude_EPSG4326']
data_filtered.columns

Index(['Sedimentary_Dictionary', 'Igneous_Dictionary',
       'Metamorphic_Dictionary', 'Seismic_LAB_Priestley', 'Seismic_Moho',
       'Gravity_GOCE_ShapeIndex', 'Geology_Paleolatitude_Period_Minimum',
       'Terrane_Proximity', 'Geology_PassiveMargin_Proximity',
       'Geology_BlackShale_Proximity', 'Geology_Fault_Proximity',
       'Gravity_Bouguer', 'Gravity_Bouguer_HGM',
       'Gravity_Bouguer_UpCont30km_HGM', 'Gravity_Bouguer_HGM_Worms_Proximity',
       'Gravity_Bouguer_UpCont30km_HGM_Worms_Proximity', 'Magnetic_HGM',
       'Magnetic_LongWavelength_HGM', 'Magnetic_HGM_Worms_Proximity',
       'Magnetic_LongWavelength_HGM_Worms_Proximity',
       'Geology_Lithology_Majority_Igneous_Extrusive',
       'Geology_Lithology_Majority_Igneous_Intrusive_Felsic',
       'Geology_Lithology_Majority_Igneous_Intrusive_Mafic',
       'Geology_Lithology_Majority_Metamorphic_Gneiss',
       'Geology_Lithology_Majority_Metamorphic_Gneiss_Paragneiss',
       'Geology_Lithology_Majority_Metamo

Adds the latitudes to the datacube to make train, validation, and test splits

In [16]:
te_df, tr_df, splits = utils.get_spatial_cross_val_idx(data_filtered)

In [17]:
hist_gbm_classifier = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=70,              # Number of boosting iterations (equivalent to n_estimators)
    max_depth=6,              # Maximum tree depth
    min_samples_leaf=48,      # Minimum samples required for a leaf node
    max_leaf_nodes=64,        # Maximum number of leaf nodes
    verbose=1                 # Show progress bars
)

In [18]:
for i, (train_index, val_index) in enumerate(splits):
    print(f"  Train: groups={np.unique(tr_df.iloc[train_index.tolist()]['group'].tolist())}")
    X_train = tr_df.iloc[train_index.tolist()].drop(columns=['target','Latitude_EPSG4326','group'])
    y_train = tr_df.iloc[train_index.tolist()]['target']
    hist_gbm_classifier.fit(X_train, y_train, sample_weight=499*y_train.astype('int')+1)

    print(f"  Val: groups={np.unique(tr_df.iloc[val_index.tolist()]['group'].tolist())}")
    X_val = tr_df.iloc[val_index.tolist()].drop(columns=['target','Latitude_EPSG4326','group'])
    y_val = tr_df.iloc[val_index.tolist()]['target']
    y_pred_val = hist_gbm_classifier.predict(X_val)

    auc_score = roc_auc_score(y_val, y_pred_val)
    print(f"Fold {i}: valid. AUC = {auc_score}")

  Train: groups=[1 2 3 4]
Binning 1.961 GB of training data: 2.796 s
Binning 0.218 GB of validation data: 0.236 s
Fitting gradient boosted rounds:
[1/70] 1 tree, 59 leaves, max depth = 6, train loss: 0.63025, val loss: 0.63200, in 0.400s
[2/70] 1 tree, 58 leaves, max depth = 6, train loss: 0.57975, val loss: 0.58299, in 0.409s
[3/70] 1 tree, 62 leaves, max depth = 6, train loss: 0.53715, val loss: 0.54153, in 0.681s
[4/70] 1 tree, 61 leaves, max depth = 6, train loss: 0.50030, val loss: 0.50577, in 0.705s
[5/70] 1 tree, 62 leaves, max depth = 6, train loss: 0.46825, val loss: 0.47446, in 0.482s
[6/70] 1 tree, 62 leaves, max depth = 6, train loss: 0.44051, val loss: 0.44779, in 0.604s
[7/70] 1 tree, 62 leaves, max depth = 6, train loss: 0.41712, val loss: 0.42498, in 0.411s
[8/70] 1 tree, 62 leaves, max depth = 6, train loss: 0.39616, val loss: 0.40506, in 0.386s
[9/70] 1 tree, 63 leaves, max depth = 6, train loss: 0.37701, val loss: 0.38620, in 0.468s
[10/70] 1 tree, 63 leaves, max dep

In [19]:
X_train = tr_df.drop(columns=['target','Latitude_EPSG4326','group'])
y_train = tr_df['target']
y_pred_train = hist_gbm_classifier.predict(X_train)
auc_score = roc_auc_score(y_train, y_pred_train)
print(f"Train AUC: {auc_score}")

X_test = te_df.drop(columns=['target','Latitude_EPSG4326','group'])
y_test = te_df['target']
y_pred_test = hist_gbm_classifier.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred_test)
print(f"Test AUC: {auc_score}")

X_all = data_filtered.drop(columns=['target','Latitude_EPSG4326'])
y_all = data_filtered['target']
y_pred_all = hist_gbm_classifier.predict(X_all)
auc_score = roc_auc_score(y_all, y_pred_all)
print(f"All AUC: {auc_score}")

Train AUC: 0.9657806933309862
Test AUC: 0.9545091232889482
All AUC: 0.9643422736338094
